# Calculate yield anomalies and plot them
- Script to take the processed crop yield data and calculate crop yield anomalies. The script calculates three different types of anomalies to enable sensitivity analyses, and writes out new objects accordingly
- In a second step these yield anomalies are written out to gridded netCDF objects for ease of use in other applications as this is the standard format for other gridded yield products
- These yield anomalies are finally also written to the original dataframe so that the data can be accessed either as a dataframe or as a gridded product


In [1]:
from scipy import signal
from scipy import ndimage
import netCDF4
import os, json
import warnings
import regionmask
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf
from tools import CreateLinkAdmin
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
#define a moving average function
def moving_average(a, smooth) :
    n=smooth*2+1
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [3]:
#read in the crop yield data
df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
#read in the FEWS admins
shape = gpd.read_file('./data/shapefile/adm_current.shp').to_crs("EPSG:4326")
#merge the two
df = df.merge(shape[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
df['admin'] = df['fnid'].apply(lambda x: x[2:8])
cols = ['fnid','country','admin','admin1','admin2','product','season','harvest_month','harvest_year','indicator','value']
df = df[cols]

#read in the crop yield data at resolution adm1 and adm0
df0 = pd.read_hdf('./data/crop/adm0_crop_production_ALL.hdf')
df1 = pd.read_hdf('./data/crop/adm1_crop_production_ALL.hdf')
#read in the FEWS admins
shape1 = gpd.read_file('./data/shapefile/adm1_current.shp').to_crs("EPSG:4326")
#merge the two admin 1s
df1 = df1.merge(shape1[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
df1 = df1.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
df1['admin'] = df1['fnid'].apply(lambda x: x[2:8])
cols = ['fnid','country','admin','admin1','product','season','harvest_month','harvest_year','indicator','value']
df1 = df1[cols]

### Calculate anomalies and grid
Below is the main portion of the script, where anomalies are calculated and written out as both a table and as a gridded netCDF product

In [4]:
#use FNIDs and years to create an empty dataframe
fnids = df.fnid.unique()
fnids1 = df1.fnid.unique()
yrs = np.sort(df.harvest_year.unique())
dfEmpty = pd.DataFrame(columns = yrs, index=fnids,dtype=float)
dfEmpty1 = pd.DataFrame(columns = yrs, index=fnids1,dtype=float)

#get a table of FNIDs and geometries
fnGeos = shape[['FNID','geometry']].drop_duplicates()
fnGeos.set_index('FNID',inplace=True)

#get a table of FNIDs and geometries for admin 1
fnGeos1 = shape1[['FNID','geometry']].drop_duplicates()
fnGeos1.set_index('FNID',inplace=True)

#make an empty grid
lon = np.arange(-20,55,0.25)
lat = np.arange(40,-40,-0.25)

cps = {
    'maize1':{'Somalia':['Maize','Deyr'],
            'Malawi':['Maize','Main'],
            'Kenya':['Maize','Short'],
            'Burkina Faso':['Maize','Main'],
            'Mali':['Maize','Main'],
            'Chad':['Maize','Main'],
            'South Africa':['Maize (Yellow)','Summer'],
            'Niger':['Maize','Main season'],
            'Zambia':['Maize','Annual'],
             'Angola':['Maize','Main'],
              'Mozambique':['Maize','Main harvest']
             },
    
    'maize2':{'Somalia':['Maize','Gu'],
            'Kenya':['Maize','Long'],
             'South Africa':['Maize (White)':'Summer']},
    
    'wheat':{'Kenya':['Wheat','Annual'],
            'Mali':['Wheat','Main'],
            'Chad':['Wheat','Main'],
            'South Africa':['Wheat','Winter']}
}

dfPrd = dfEmpty.copy()
dfArea = dfEmpty.copy()
dfYld = dfEmpty.copy()
dfYldGauAbs = dfEmpty.copy()
dfYldSm5 = dfEmpty.copy()
dfYldGau = dfEmpty.copy()
dfPrdGau = dfEmpty.copy()
dfAreaGau = dfEmpty.copy()

dfPrd1 = dfEmpty1.copy()
dfArea1 = dfEmpty1.copy()
dfYld1 = dfEmpty1.copy()
dfYldGauAbs1 = dfEmpty1.copy()
dfYldSm51 = dfEmpty1.copy()
dfYldGau1 = dfEmpty1.copy()
dfPrdGau1 = dfEmpty1.copy()
dfAreaGau1 = dfEmpty1.copy()

dfPrd0 = dfEmpty.copy()
dfArea0 = dfEmpty.copy()
dfYld0 = dfEmpty.copy()
dfYldGauAbs0 = dfEmpty.copy()
dfYldSm50 = dfEmpty.copy()
dfYldGau0 = dfEmpty.copy()
dfPrdGau0 = dfEmpty.copy()
dfAreaGau0 = dfEmpty.copy()

for icr in ['maize1','maize2','wheat']:
    ncPath = '/Users/wanders7/Documents/Code/Project/NASA_GSCD/gscd/data/gridded/admFull_'+icr+'.nc'
    ncf = netCDF4.Dataset(ncPath,'w',format='NETCDF4')
    #create dimensions                            
    ncf.createDimension('lon',lon.size)
    ncf.createDimension('lat',lat.size)
    ncf.createDimension('yr',yrs.size)
    #create variables
    ncf.createVariable('latitude','f4',('lat'))
    ncf.createVariable('longitude','f4',('lon'))
    ncf.createVariable('year','i8',('yr'))
    #fill variables lat/lon
    ncf.variables['latitude'][:]=lat
    ncf.variables['longitude'][:]=lon    
    ncf.variables['year'][:]=yrs
    #create the yield variables and objects
    ncf.createVariable('yieldAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('prodAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('areaAnom_pct','f4',('yr','lat','lon'))
    ncf.createVariable('yieldAnom_abs','f4',('yr','lat','lon'))
    ncf.createVariable('yield','f4',('yr','lat','lon'))
    ncf.createVariable('production','f4',('yr','lat','lon'))
    ncf.createVariable('area','f4',('yr','lat','lon'))

    ncPath1 = '/Users/wanders7/Documents/Code/Project/NASA_GSCD/gscd/data/gridded/adm1_'+icr+'.nc'
    nc1 = netCDF4.Dataset(ncPath1,'w',format='NETCDF4')
    #create dimensions                            
    nc1.createDimension('lon',lon.size)
    nc1.createDimension('lat',lat.size)
    nc1.createDimension('yr',yrs.size)
    #create variables
    nc1.createVariable('latitude','f4',('lat'))
    nc1.createVariable('longitude','f4',('lon'))
    nc1.createVariable('year','i8',('yr'))
    #fill variables lat/lon
    nc1.variables['latitude'][:]=lat
    nc1.variables['longitude'][:]=lon    
    nc1.variables['year'][:]=yrs
    #create the yield variables and objects
    nc1.createVariable('yieldAnom_pct','f4',('yr','lat','lon'))
    nc1.createVariable('prodAnom_pct','f4',('yr','lat','lon'))
    nc1.createVariable('areaAnom_pct','f4',('yr','lat','lon'))
    nc1.createVariable('yieldAnom_abs','f4',('yr','lat','lon'))
    nc1.createVariable('yield','f4',('yr','lat','lon'))
    nc1.createVariable('production','f4',('yr','lat','lon'))
    nc1.createVariable('area','f4',('yr','lat','lon'))
    
    area = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prd = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yld = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnomAbs = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prdAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
    areaAnom = np.ones([yrs.size,lat.size,lon.size])*np.nan
 
    area1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prd1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yld1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnom1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    yldAnomAbs1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    prdAnom1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    areaAnom1 = np.ones([yrs.size,lat.size,lon.size])*np.nan
    
    for icx in cps[icr].keys():
        ipx = cps[icr][icx][0]
        isx = cps[icr][icx][1]
        ifnds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)].fnid.unique()
        ifnds1 = df1.loc[(df1['product']==ipx)&(df1.country==icx)&(df1.season==isx)].fnid.unique()
        
        for ifnx in ifnds:
            shp = fnGeos.loc[ifnx].geometry
            
            ylds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='yield')].sort_values(by='harvest_year')
            
            prds = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='production')].sort_values(by='harvest_year')

            areas = df.loc[(df['product']==ipx)&(df.country==icx)&(df.season==isx)&
                   (df.fnid==ifnx)&(df.indicator=='area')].sort_values(by='harvest_year')

            #check for duplicate values, which indicates multiple production systems. 
            # If present, sum the area and production and recalculate yield
            if ((np.size(ylds['harvest_year'][ylds['harvest_year'].duplicated()])>0)|
                (np.size(prds['harvest_year'][prds['harvest_year'].duplicated()])>0)|
                (np.size(areas['harvest_year'][areas['harvest_year'].duplicated()])>0)):
                cols = ['fnid','country','admin','admin1','admin2','product','season',
                        'harvest_month','harvest_year','indicator','value']
                #group area and production
                areas = areas[cols].groupby(cols[:-1]).sum()
                prds = prds[cols].groupby(cols[:-1]).sum()
                #reindex
                indexCols = cols[:-2]
                prds = prds.reset_index()
                prds.set_index(indexCols,inplace=True)
                areas = areas.reset_index()
                areas.set_index(indexCols,inplace=True)
                #write to the yield df
                ylds = prds['value']/areas['value']
                ylds=ylds.reset_index()
                prds=prds.reset_index()
                areas=areas.reset_index()
                ylds['indicator']='yield'
                
            if ylds.value.size<5:
                print('fewer than 5 years for '+ifnx+', '+ipx+', '+isx)
                continue
            
            dfPrd.loc[ifnx][prds.harvest_year] = prds.value
            dfArea.loc[ifnx][areas.harvest_year] = areas.value
            dfYld.loc[ifnx][ylds.harvest_year] = ylds.value
            
            iYld = dfYld.loc[ifnx][ylds.harvest_year].values.astype(float)
            iyldAbs = dfYld.loc[ifnx][ylds.harvest_year].values.astype(float)
            iprdAbs = dfPrd.loc[ifnx][prds.harvest_year].values.astype(float)
            iareaAbs = dfArea.loc[ifnx][areas.harvest_year].values.astype(float)

            #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
            yldCount = np.copy(iYld)
            yldCount[~np.isnan(yldCount)]=1
            yldCount[np.isnan(yldCount)]=0
            iYld[np.isnan(iYld)]=0

            prdCount = np.copy(iprdAbs)
            prdCount[~np.isnan(prdCount)]=1
            prdCount[np.isnan(prdCount)]=0
            iprdAbs[np.isnan(iprdAbs)]=0
            
            areaCount = np.copy(iareaAbs)
            areaCount[~np.isnan(areaCount)]=1
            areaCount[np.isnan(areaCount)]=0
            iareaAbs[np.isnan(iareaAbs)]=0
            
            #divide by the count to re-weight based on the # of values going into each smoothing filter
            yldSm5Exp = moving_average(np.array(iYld),2)/moving_average(np.array(yldCount),2)
            
            yldGauExp = ndimage.gaussian_filter1d(iYld,3)/ndimage.gaussian_filter1d(yldCount,3)
            prdGauExp = ndimage.gaussian_filter1d(iprdAbs,3)/ndimage.gaussian_filter1d(prdCount,3)
            areaGauExp = ndimage.gaussian_filter1d(iareaAbs,3)/ndimage.gaussian_filter1d(areaCount,3)
            
            stYldGauAbs = (iYld-yldGauExp)
            #percent yield anom = (yld obs - yld exp)/yld exp
            stYldGau = (iYld-yldGauExp)/yldGauExp
            stPrdGau = (iprdAbs-prdGauExp)/prdGauExp
            stAreaGau = (iareaAbs-areaGauExp)/areaGauExp
            stYldSm5 = np.array(iYld[2:-2]-yldSm5Exp)/yldSm5Exp
            
            #add nans back in
            stPrdGau[np.isnan(dfPrd.loc[ifnx][prds.harvest_year].values.astype(float))]=np.nan
            stAreaGau[np.isnan(dfArea.loc[ifnx][areas.harvest_year].values.astype(float))]=np.nan
            stYldGau[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values.astype(float))]=np.nan
            stYldSm5[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values[2:-2].astype(float))]=np.nan
            stYldGauAbs[np.isnan(dfYld.loc[ifnx][ylds.harvest_year].values.astype(float))]=np.nan
            
            #write the empty data frames
            dfYldGauAbs.loc[ifnx][ylds.harvest_year.values] = stYldGauAbs
            dfYldGau.loc[ifnx][ylds.harvest_year.values] = stYldGau
            dfPrdGau.loc[ifnx][prds.harvest_year.values] = stPrdGau
            dfAreaGau.loc[ifnx][areas.harvest_year.values] = stAreaGau
            dfYldSm5.loc[ifnx][ylds.harvest_year.values][2:-2] = stYldSm5
            
            #find the location mask
            msk = np.array(regionmask.Regions([fnGeos.loc[ifnx].geometry]).mask(lon,lat))
            #assign the yield time series to the relevant grid points based on the mask
            prdAnom.T[msk.T==0,...] = dfPrdGau.loc[ifnx].values
            areaAnom.T[msk.T==0,...] = dfAreaGau.loc[ifnx].values
            yldAnom.T[msk.T==0,...] = dfYldGau.loc[ifnx].values
            yldAnomAbs.T[msk.T==0,...] = dfYldGauAbs.loc[ifnx].values
            
            yld.T[msk.T==0,...] = dfYld.loc[ifnx].values
            prd.T[msk.T==0,...] = dfPrd.loc[ifnx].values
            area.T[msk.T==0,...] = dfArea.loc[ifnx].values 
            
            #Take the gaussian yield anomalies and write them to the original dataframe
            yldAnomGauDF = ylds.copy(deep=True) #copy the yields
            yldAnomGauDF['value']= np.nan #set values to nan
            yldAnomGauDF['value']= stYldGau#write in the new values
            yldAnomGauDF['indicator'] = 'frac_yld_anom_gauss'
            prdAnomGauDF = prds.copy(deep=True) #copy the yields
            prdAnomGauDF['value']= np.nan #set values to nan
            prdAnomGauDF['value']= stPrdGau#write in the new values
            prdAnomGauDF['indicator'] = 'frac_prd_anom_gauss'
            areaAnomGauDF = areas.copy(deep=True) #copy the yields
            areaAnomGauDF['value']= np.nan #set values to nan
            areaAnomGauDF['value']= stAreaGau#write in the new values
            areaAnomGauDF['indicator'] = 'frac_area_anom_gauss'
            df = df.append(areaAnomGauDF).append(prdAnomGauDF).append(yldAnomGauDF)

        for ifnx in ifnds1:
            shp = fnGeos1.loc[ifnx].geometry
            
            ylds1 = df1.loc[(df1['product']==ipx)&(df1.country==icx)&(df1.season==isx)&
                   (df1.fnid==ifnx)&(df1.indicator=='yield')].sort_values(by='harvest_year')
            
            prds1 = df1.loc[(df1['product']==ipx)&(df1.country==icx)&(df1.season==isx)&
                   (df1.fnid==ifnx)&(df1.indicator=='production')].sort_values(by='harvest_year')

            areas1 = df1.loc[(df1['product']==ipx)&(df1.country==icx)&(df1.season==isx)&
                   (df1.fnid==ifnx)&(df1.indicator=='area')].sort_values(by='harvest_year')

            #check for duplicate values, which indicates multiple production systems. 
            # If present, sum the area and production and recalculate yield
            if ((np.size(ylds1['harvest_year'][ylds1['harvest_year'].duplicated()])>0)|
                (np.size(prds1['harvest_year'][prds1['harvest_year'].duplicated()])>0)|
                (np.size(areas1['harvest_year'][areas1['harvest_year'].duplicated()])>0)):
                cols = ['fnid','country','admin','admin1','product','season',
                        'harvest_month','harvest_year','indicator','value']
                #group area and production
                areas1 = areas1[cols].groupby(cols[:-1]).sum()
                prds1 = prds1[cols].groupby(cols[:-1]).sum()
                #reindex
                indexCols = cols[:-2]
                prds1 = prds1.reset_index()
                prds1.set_index(indexCols,inplace=True)
                areas1 = areas1.reset_index()
                areas1.set_index(indexCols,inplace=True)
                #write to the yield df
                ylds1 = prds1['value']/areas1['value']
                ylds1=ylds1.reset_index()
                prds1=prds1.reset_index()
                areas1=areas1.reset_index()
                ylds1['indicator']='yield'
                
            if ylds1.value.size<5:
                print('fewer than 5 years for '+ifnx+', '+ipx+', '+isx)
                continue
            
            dfPrd1.loc[ifnx][prds1.harvest_year] = prds1.value
            dfArea1.loc[ifnx][areas1.harvest_year] = areas1.value
            dfYld1.loc[ifnx][ylds1.harvest_year] = ylds1.value
            
            iYld1 = dfYld1.loc[ifnx][ylds1.harvest_year].values.astype(float)
            iyldAbs1 = dfYld1.loc[ifnx][ylds1.harvest_year].values.astype(float)
            iprdAbs1 = dfPrd1.loc[ifnx][prds1.harvest_year].values.astype(float)
            iareaAbs1 = dfArea1.loc[ifnx][areas1.harvest_year].values.astype(float)

            #To account for a missing values in locations we can drop and re-weight the remaining values by counting non-zero values        
            yldCount1 = np.copy(iYld1)
            yldCount1[~np.isnan(yldCount1)]=1
            yldCount1[np.isnan(yldCount1)]=0
            iYld1[np.isnan(iYld1)]=0

            prdCount1 = np.copy(iprdAbs1)
            prdCount1[~np.isnan(prdCount1)]=1
            prdCount1[np.isnan(prdCount1)]=0
            iprdAbs1[np.isnan(iprdAbs1)]=0
            
            areaCount1 = np.copy(iareaAbs1)
            areaCount1[~np.isnan(areaCount1)]=1
            areaCount1[np.isnan(areaCount1)]=0
            iareaAbs1[np.isnan(iareaAbs1)]=0
            
            #divide by the count to re-weight based on the # of values going into each smoothing filter
            yldSm5Exp1 = moving_average(np.array(iYld1),2)/moving_average(np.array(yldCount1),2)
            
            yldGauExp1 = ndimage.gaussian_filter1d(iYld1,3)/ndimage.gaussian_filter1d(yldCount1,3)
            prdGauExp1 = ndimage.gaussian_filter1d(iprdAbs1,3)/ndimage.gaussian_filter1d(prdCount1,3)
            areaGauExp1 = ndimage.gaussian_filter1d(iareaAbs1,3)/ndimage.gaussian_filter1d(areaCount1,3)
            
            stYldGauAbs1 = (iYld1-yldGauExp1)
            #percent yield anom = (yld obs - yld exp)/yld exp
            stYldGau1 = (iYld1-yldGauExp1)/yldGauExp1
            stPrdGau1 = (iprdAbs1-prdGauExp1)/prdGauExp1
            stAreaGau1 = (iareaAbs1-areaGauExp1)/areaGauExp1
            stYldSm51 = np.array(iYld1[2:-2]-yldSm5Exp1)/yldSm5Exp1
            
            #add nans back in
            stPrdGau1[np.isnan(dfPrd1.loc[ifnx][prds1.harvest_year].values.astype(float))]=np.nan
            stAreaGau1[np.isnan(dfArea1.loc[ifnx][areas1.harvest_year].values.astype(float))]=np.nan
            stYldGau1[np.isnan(dfYld1.loc[ifnx][ylds1.harvest_year].values.astype(float))]=np.nan
            stYldSm51[np.isnan(dfYld1.loc[ifnx][ylds1.harvest_year].values[2:-2].astype(float))]=np.nan
            stYldGauAbs1[np.isnan(dfYld1.loc[ifnx][ylds1.harvest_year].values.astype(float))]=np.nan
            
            #write the empty data frames
            dfYldGauAbs1.loc[ifnx][ylds1.harvest_year.values] = stYldGauAbs1
            dfYldGau1.loc[ifnx][ylds1.harvest_year.values] = stYldGau1
            dfPrdGau1.loc[ifnx][prds1.harvest_year.values] = stPrdGau1
            dfAreaGau1.loc[ifnx][areas1.harvest_year.values] = stAreaGau1
            dfYldSm51.loc[ifnx][ylds1.harvest_year.values][2:-2] = stYldSm51
            
            #find the location mask
            msk = np.array(regionmask.Regions([fnGeos1.loc[ifnx].geometry]).mask(lon,lat))
            #assign the yield time series to the relevant grid points based on the mask
            prdAnom1.T[msk.T==0,...] = dfPrdGau1.loc[ifnx].values
            areaAnom1.T[msk.T==0,...] = dfAreaGau1.loc[ifnx].values
            yldAnom1.T[msk.T==0,...] = dfYldGau1.loc[ifnx].values
            yldAnomAbs1.T[msk.T==0,...] = dfYldGauAbs1.loc[ifnx].values
            
            yld1.T[msk.T==0,...] = dfYld1.loc[ifnx].values
            prd1.T[msk.T==0,...] = dfPrd1.loc[ifnx].values
            area1.T[msk.T==0,...] = dfArea1.loc[ifnx].values 
            
            #Take the gaussian yield anomalies and write them to the original dataframe
            yldAnomGauDF1 = ylds1.copy(deep=True) #copy the yields
            yldAnomGauDF1['value']= np.nan #set values to nan
            yldAnomGauDF1['value']= stYldGau1 #write in the new values
            yldAnomGauDF1['indicator'] = 'frac_yld_anom_gauss'
            prdAnomGauDF1 = prds1.copy(deep=True) #copy the yields
            prdAnomGauDF1['value']= np.nan #set values to nan
            prdAnomGauDF1['value']= stPrdGau1 #write in the new values
            prdAnomGauDF1['indicator'] = 'frac_prd_anom_gauss'
            areaAnomGauDF1 = areas1.copy(deep=True) #copy the yields
            areaAnomGauDF1['value']= np.nan #set values to nan
            areaAnomGauDF1['value']= stAreaGau1 #write in the new values
            areaAnomGauDF1['indicator'] = 'frac_area_anom_gauss'
            df1 = df1.append(areaAnomGauDF1).append(prdAnomGauDF1).append(yldAnomGauDF1)
            
    #write the yield objects into the netCDF
    ncf['yield'][:] = yld
    ncf['production'][:] = prd
    ncf['area'][:] = area
    ncf['yieldAnom_pct'][:] = yldAnom
    ncf['prodAnom_pct'][:] = prdAnom
    ncf['areaAnom_pct'][:] = areaAnom
    ncf['yieldAnom_abs'][:] = yldAnomAbs
    
    ncf.close();del ncf
    
    #write the yield objects into the netCDF
    nc1['yield'][:] = yld1
    nc1['production'][:] = prd1
    nc1['area'][:] = area1
    nc1['yieldAnom_pct'][:] = yldAnom1
    nc1['prodAnom_pct'][:] = prdAnom1
    nc1['areaAnom_pct'][:] = areaAnom1
    nc1['yieldAnom_abs'][:] = yldAnomAbs1
    
    nc1.close();del nc1

save_hdf('./data/crop/adm_crop_production_anomalies_ALL.hdf', df)
save_hdf('./data/crop/adm1_crop_production_anomalies_ALL.hdf', df1)

SyntaxError: invalid syntax (963293841.py, line 36)